In [5]:
import numpy as np
import pandas as pd
import panel as pn
import holoviews as hv
import hvplot.pandas
import param
import random
from datetime import datetime
pn.extension()

###   step1.数据处理
data = pd.read_csv('world_wide_COVID_data_daily_data.csv')
data.loc[:, 'dateRep'] = data['dateRep'].apply(lambda x: datetime.strptime(x,"%d/%m/%Y"))   ### 转换时间格式
countriesData = {}   ### 统计洲对应的国家数据
for row in data.itertuples():
    if row.continentExp not in countriesData:
        countriesData[row.continentExp] = [row.countriesAndTerritories]
    elif row.countriesAndTerritories not in countriesData[row.continentExp]:
        countriesData[row.continentExp].append(row.countriesAndTerritories)
        
###   step2.定义面板这个类     
class MyPanel(param.Parameterized):
    continents = param.ListSelector(default=['Asia'], objects=countriesData.keys())   ### 大洲选择
    countries = param.ListSelector(default=['China'], objects=countriesData['Asia'])   ### 国家选择
    months = param.Range(default=(2, 3), bounds=(1, 12))   ### 月份
    dataType  = param.Selector(default="cases", objects=["cases", "deaths"])   ### 展示数据类型
    
    _countries = countriesData   ### 国家数据
    
    @param.depends('continents', watch=True)   ### 大洲变化时更新国家
    def _update_countries(self):
        countries = []
        for i in self.continents:
            countries= countries + self._countries[i]
        self.param['countries'].objects = countries
        self.countries = [countries[0]]

    @param.depends('countries','months','dataType')    ### 国家，月份，数据类型变化时更新图和表格
    def view(self):
        start = self.months[0]
        end = self.months[1]
        graph_data = data[(data['continentExp'].isin(self.continents))&(data['countriesAndTerritories'].isin(self.countries))&(data['month']<=end)&(data['month']>=start)]   ###   根据参数筛选数据
        timeline = graph_data.hvplot(x='dateRep', y=self.dataType, xlabel='Date', rot=90, sort_date=True, by='countriesAndTerritories')   ### 数据图
        table = graph_data.hvplot.table(sortable=True)   ### 表格
        return pn.Column(timeline,table)

###   step3.新建一个面板把内容组合在一起
viewer = MyPanel(name='Setting')
pn.Row(pn.Param(viewer.param, widgets={'dataType': pn.widgets.RadioButtonGroup}), viewer.view)

Row
    [0] Param(MyPanel, name='Setting', widgets={'dataType': <...})
    [1] ParamMethod(method)